In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import bs4
import re
from multiprocessing import Pool, Lock, Value
from time import sleep
import os
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem.snowball import SnowballStemmer 
import json
from tqdm import tqdm

In [2]:
queries = pd.read_csv('queries.numerate.txt', sep='	', header=None)
urls = pd.read_csv('urls.numerate.txt', sep='	', header=None)
urls.index = urls[1]

In [10]:
docnames = []

listdir = os.listdir('content/content/')

for d in listdir:
    listdocs = os.listdir('content/content/' + d)
    listdocs.sort()
    docnames += list(map(lambda x: d + '/' + x, listdocs))
docnames[-5:]

['20170717/doc.4950.dat',
 '20170717/doc.4951.dat',
 '20170717/doc.4952.dat',
 '20170717/doc.4953.dat',
 '20170717/doc.4954.dat']

digits???

In [25]:
def doc2words(docname):
    with open('content/content/' + docname, errors='ignore') as read_file:
        lines = list(read_file)
    url = lines[0].strip()
    html = "".join(lines[1:])
    soup = BeautifulSoup(html)
    doc_id = urls.at[url, 0]
    if soup.text:
        text = re.sub(r'[^A-Za-zА-Яа-я\.\-]+', ' ', soup.text)
    else:
        text = ""
    if soup.title:
        if soup.title.text:
            title = re.sub(r'[^A-Za-zА-Яа-я\.\-]+', ' ', soup.title.text)
        else:
            title = ""
    else:
        title = ""

    return (title + text, doc_id)

In [26]:
mutex = Lock()
n_processed = Value('i', 0)

def func_wrapper(docname):
    res = doc2words(docname) 
    with mutex:
        # в этом блоке можно безопасно менять общие объекты для процессов
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

with Pool(processes=10) as pool:
    docs = pool.map(func_wrapper, docnames)

38110 objects are processed...

In [33]:
docs_text = list(map(lambda x: x[0], docs))
docs_id = list(map(lambda x: x[1], docs))

doc_df = pd.DataFrame({'id': docs_id, 'text': docs_text})
doc_df = doc_df.sort_values(by=['id'])
doc_df.head()

,id,text
21700,1,Хорватия Хорватия EVA.RU Вход Европа Черногори...
22010,2,Три недели в Европе. Начало Три недели в Европ...
21232,3,Как правильно сделать визу в Польшу шенген Q A...
20062,4,Новости Новости О нас Отрасли Страны Новости. ...
20345,5,Аллергия на Магические Грибы fb КулЛиб - Класс...


In [34]:
doc_df.to_csv("text_relevance.csv")

### index + tf-idf

In [3]:
doc_df = pd.read_csv("text_relevance.csv")

In [17]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words |= set(stopwords.words('russian'))
with open('stop_words.txt', "r") as rf:
    lines = [line.strip() for line in rf]
stop_words |= set(lines)
stop_words |= set(['.', '...', '-'])

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
doc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38114 entries, 0 to 38113
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  38114 non-null  int64 
 1   id          38114 non-null  int64 
 2   text        38114 non-null  object
dtypes: int64(2), object(1)
memory usage: 893.4+ KB


In [7]:
doc_df.index = doc_df['id']

In [8]:
doc_df.head()

,Unnamed: 0,id,text
id,,,
1,21700,1,Хорватия Хорватия EVA.RU Вход Европа Черногори...
2,22010,2,Три недели в Европе. Начало Три недели в Европ...
3,21232,3,Как правильно сделать визу в Польшу шенген Q A...
4,20062,4,Новости Новости О нас Отрасли Страны Новости. ...
5,20345,5,Аллергия на Магические Грибы fb КулЛиб - Класс...


{term1: [[doc_id1, term_freq1], [doc_id2, term_freq2], ...], term2: ...}

In [9]:
index = {}

In [10]:
i = 0
stemmer = SnowballStemmer("russian") 
for id, doc in tqdm(doc_df.iterrows(), total=20000, position=0):
    i += 1
    if i == 20000:
        break
    terms = doc['text'].split()
    terms = [t for t in terms if not t in stop_words]
    terms = [stemmer.stem(term) for term in terms]
    for term in terms:
        if term in index.keys():
            doc_id, num = index[term][-1]
            if doc_id == id:
                index[term][-1][1] += 1
            else:
                index[term].append([id, 1])
        else:
            index[term] = [[id, 1]]

 75%|███████▍  | 14925/20000 [2:12:13<23:49,  3.55it/s]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|█████████▉| 19998/20000 [2:57:27<00:00,  5.33it/s]  

In [ ]:
with open('index.json', 'w') as f:
    json.dump(index, f)

In [3]:
with open('index.json', 'r') as f:
    index = json.load(f)

In [19]:
queries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       399 non-null    int64 
 1   1       399 non-null    object
dtypes: int64(1), object(1)
memory usage: 6.4+ KB


In [53]:
QueryId = []

for i in range(1, 400):
    QueryId += [i] * 10
    
# size = 10 * 399

TODO !!! lowercase

* term-frequency

$tf = \frac{n_t}{\sum{n_k}}$

$n_t$ - число вхождений слова t в документ

$\sum{n_k}$ - общее число слов в данном документе

* inverse document frequency

$idf = log \frac{|D|}{|d_i : t \in d_i|}$ 

$|D|$ - число документов в коллекции

$|d_i : t \in d_i|$ - число документов из коллекции D, в которых встречается t

text[0] - QueryId

text[1] - queries text

docs_list = [[doc_id1, term_freq1], [doc_id2, term_freq2], ...]

In [49]:
stemmer = SnowballStemmer("russian") 

D = len(index)
docsId = np.ones(10 * 398, dtype=int)

for id, text in tqdm(queries.iterrows(), total=399, position=0):
    words = text[1].split()
    words = [w for w in words if not w in stop_words]
    words = [stemmer.stem(word) for word in words]
    
    words_info = [] # list of lists with docs numbers and number of word entries
    
    for i in range(len(words)):
        word = words[i]
        if word in index.keys():
            words_info.append(index[word])
        else:
            words_info.append([])
    # id of all docs which contain one of query's word
    all_docs = list(set([doc_info[0] for word_info in words_info 
                         for doc_info in word_info]))
    # word's counter for each document
    docs_cnt = {el:0 for el in all_docs}
    
    # computing sum tf-idf for each document
    for word_info in words_info:
        if word_info:
            idf = np.log(D / len(word_info))
        else:
            idf = 0
        for doc_info in word_info: # doc_info = [doc_id, term_freq]
            docs_cnt[doc_info[0]] += doc_info[1] * idf

    # sort docs by it's frequency for query's words
    docs_cnt = [item for item in docs_cnt.items()]
    docs_cnt = sorted(docs_cnt, key=lambda x: x[1], reverse=True) # [(doc_id1, freq1), (doc_id2, freq2), ...]
    i = 0
    
    # docsId[id - 1 + i]
    for item in docs_cnt: # item[0] = doc_id
        docsId[(text[0] - 1) * 10 + i] = item[0]
        i += 1
        if i == 10:
            break

100%|██████████| 399/399 [00:09<00:00, 40.89it/s]


In [55]:
result_df = pd.DataFrame({
    'QueryId' : QueryId,
    'DocumentId' : docsId
})

In [59]:
result_df.head()

,QueryId,DocumentId
0,1,9799
1,1,1849
2,1,247
3,1,2270
4,1,14475


In [62]:
result_df.to_csv('result.txt', index=False)